## Philadelphia Housing Analysis - ETL Notebook

## 1.) Dependencies and setup

In [1]:
# 1a.) Define imports
import pandas as pd

# 1b.) Define source data location and source file 
source_file_loc = 'source_data/'
data_file = 'opa_properties_public_2016_2020.csv'

## 2.) Read in source dataset

In [2]:
# 2a.) Read in source dataset using pd.read_csv
housing_df_raw = pd.read_csv(source_file_loc + data_file,low_memory=False)

In [3]:
# 2b.) Verify df load by outputting head
housing_df_raw.head()

,objectid,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,...,unfinished,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,lat,lng
0,30283185,NaN,H,70' N FROM NS OF,586035,O50,ROW 3 STY MASONRY,1,Single Family,11.0,...,NaN,NaN,NaN,I,1875,Y,19147.0,RM1,-75.164191,39.943765
1,30788703,NaN,NaN,83.573' W 41ST,382467,CA0,APTS 5-50 UNITS MASONRY,2,Multi Family,110.0,...,NaN,NaN,NaN,I,1915,NaN,19104.0,RM1,-75.207255,39.975039
2,30788704,NaN,NaN,101.573' W 41ST,382467,CA0,APTS 5-50 UNITS MASONRY,2,Multi Family,110.0,...,NaN,NaN,NaN,I,1915,Y,19104.0,RM1,-75.207315,39.975060
3,30788705,NaN,NaN,20.667' W 42ND ST,382467,CA0,APTS 5-50 UNITS MASONRY,2,Multi Family,110.0,...,NaN,NaN,NaN,I,1915,NaN,19104.0,RSA3,-75.209677,39.976058
4,30788706,NaN,NaN,93' W 42ND ST,382467,CA0,APTS 5-50 UNITS MASONRY,2,Multi Family,110.0,...,NaN,NaN,NaN,I,1915,Y,19104.0,RSA3,-75.209910,39.976144


In [4]:
# 2c.) Verify that row count loaded matches that of source file by outputting len
len(housing_df_raw)

143152

## 3.) Remove rows out of scope for analysis

In [7]:
# 3a.) Keep only properties with category codes of Single Family, Multi-Family, Mixed Use (remove vacant lots) and residential zoning only
zoning_list = ['RM1','RM2','RM3','RM4','RMX1','RMX2','RMX3','RSA1','RSA2','RSA3','RSA4','RSA5','RSD1','RSD2','RSD3','RTA1']
category_list = ['Single Family','Multi Family','Mixed Use']
housing_df_row_filter = housing_df_raw.loc[(housing_df_raw['zoning'].isin(zoning_list) == True)&
                                           (housing_df_raw['category_code_description'].isin(category_list) == True),:]

# 3b.) Take out any records designated as "Unfinished" - unfinished field equals "U"
housing_df_row_filter = housing_df_row_filter.loc[housing_df_row_filter['unfinished'] != 'U',:]

# 3c.) Take out any records with a sale price under 1000 - assumed to be "gifts" and not relevant for a pricing analysis
housing_df_row_filter = housing_df_row_filter.loc[housing_df_row_filter['sale_price'] >= 1000,:]

# 3d.) Take out any records with number of rooms set as 0
housing_df_row_filter = housing_df_row_filter.loc[housing_df_row_filter['number_of_rooms'] > 0,:]



In [9]:
# 3e.) Confirm number of rows after filtering out rows out of scope for analysis
len(housing_df_row_filter)


68485

## 4.) Remove columns out of scope for analysis

In [10]:
# 4a.) Define list of columns to keep
columns = ['basements',
'building_code_description',
'category_code_description',
'census_tract',
'central_air',
'depth',
'exempt_building',
'exempt_land',
'exterior_condition',
'fireplaces',
'frontage',
'fuel',
'garage_spaces',
'garage_type',
'geographic_ward',
'interior_condition',
'location',
'market_value',
'market_value_date',
'number_of_bathrooms',
'number_of_bedrooms',
'number_of_rooms',
'number_stories',
'quality_grade',
'sale_date',
'sale_price',
'street_designation',
'street_direction',
'street_name',
'taxable_building',
'taxable_land',
'topography',
'total_area',
'total_livable_area',
'type_heater',
'unit',
'view_type',
'year_built',
'year_built_estimate',
'zip_code',
'zoning']

# 4b.) Create new dataframe that only contains columns in above list
housing_df_column_filter = housing_df_row_filter[columns]

# 4c.) Confirm that output df contains number of columns expected
housing_df_column_filter.head()

,basements,building_code_description,category_code_description,census_tract,central_air,depth,exempt_building,exempt_land,exterior_condition,fireplaces,...,topography,total_area,total_livable_area,type_heater,unit,view_type,year_built,year_built_estimate,zip_code,zoning
29,D,ROW 3 STY MASONRY,Single Family,241.0,N,67.0,49200,0,4.0,0.0,...,F,938.00,1344.0,A,NaN,I,1895,Y,19144.0,RSA5
31,NaN,ROW 2 STY MASONRY,Single Family,201.0,NaN,70.0,0,0,4.0,0.0,...,F,1044.00,1190.0,NaN,NaN,I,1940,Y,19140.0,RM1
32,H,ROW B/GAR 2 STY MASONRY,Single Family,281.0,N,95.5,0,0,4.0,0.0,...,F,1686.53,1633.0,B,NaN,I,1940,Y,19141.0,RSA3
37,NaN,ROW 2 STY MASONRY,Single Family,293.0,NaN,112.5,0,0,4.0,0.0,...,F,2165.62,1320.0,B,NaN,I,1940,Y,19124.0,RSA5
44,NaN,ROW 2 STY MASONRY,Single Family,62.0,NaN,79.0,0,0,4.0,0.0,...,F,1264.00,960.0,NaN,NaN,I,1920,NaN,19142.0,RM1


In [11]:
# 4d.) Confirm that number of rows is still the same as the Step 3 end result
len(housing_df_column_filter)

68485

## 5.) Reformat/manipulate columns as needed

In [14]:
# 5a.) Convert sale_date column from Excel julian date to friendly YYYY-MM-DD format
# NOTE: approach taken from https://stackoverflow.com/questions/38454403/convert-excel-style-date-with-pandas
housing_df_column_filter.loc['sale_date'] = pd.to_datetime(housing_df_column_filter.loc['sale_date'], unit='D', origin='1899-12-30')
housing_df_column_filter['sale_date'].head()

29   2020-09-28
31   2020-09-25
32   2020-09-25
37   2020-09-24
44   2020-09-24
Name: sale_date, dtype: datetime64[ns]

In [16]:
# 5b.) Zip code column currently float (has ".0" at the end), reformat as string
housing_df_column_filter['zip_code'] = housing_df_column_filter['zip_code'].astype('Int64').astype('str')
housing_df_column_filter['zip_code'].head()

<ipython-input-16-15c5598752a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_df_column_filter['zip_code'] = housing_df_column_filter['zip_code'].astype('Int64').astype('str')


29    19144
31    19140
32    19141
37    19124
44    19142
Name: zip_code, dtype: object

## 6.) Create final dataset and export csv file to be used downstream

In [17]:
# 6a.) Declare final dataframe for use downstream
housing_df_final = housing_df_column_filter
housing_df_final.head()

,basements,building_code_description,category_code_description,census_tract,central_air,depth,exempt_building,exempt_land,exterior_condition,fireplaces,...,topography,total_area,total_livable_area,type_heater,unit,view_type,year_built,year_built_estimate,zip_code,zoning
29,D,ROW 3 STY MASONRY,Single Family,241.0,N,67.0,49200,0,4.0,0.0,...,F,938.00,1344.0,A,NaN,I,1895,Y,19144,RSA5
31,NaN,ROW 2 STY MASONRY,Single Family,201.0,NaN,70.0,0,0,4.0,0.0,...,F,1044.00,1190.0,NaN,NaN,I,1940,Y,19140,RM1
32,H,ROW B/GAR 2 STY MASONRY,Single Family,281.0,N,95.5,0,0,4.0,0.0,...,F,1686.53,1633.0,B,NaN,I,1940,Y,19141,RSA3
37,NaN,ROW 2 STY MASONRY,Single Family,293.0,NaN,112.5,0,0,4.0,0.0,...,F,2165.62,1320.0,B,NaN,I,1940,Y,19124,RSA5
44,NaN,ROW 2 STY MASONRY,Single Family,62.0,NaN,79.0,0,0,4.0,0.0,...,F,1264.00,960.0,NaN,NaN,I,1920,NaN,19142,RM1


In [ ]:
# 6b.) Export cleaned df to csv to be used in other notebooks to create visualizations

cleaned_data_loc = f'{source_file_loc}housing_data_cleaned.csv'
housing_df_final.to_csv(cleaned_data_loc,sep=',',index=False)